# Task 2 LSTM

References:
- [ECG Heartbeat Classification: A Deep Transferable Representation](https://arxiv.org/pdf/1805.00794.pdf)

## Load dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score
from keras.utils import to_categorical

2023-06-08 12:51:21.521613: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## MIT-BIH  Arrhythmia Dataset

- Number of Samples: 109446
- Number of Categories: 5
- Sampling Frequency: 125Hz
- Data Source: Physionet's MIT-BIH Arrhythmia Dataset
- Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]
- Remark: All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 188.
- The final element of each row denotes the class to which that example belongs.


In [3]:
df_mitbih_train = pd.read_csv("../ecg_dataset/mitbih_train.csv", header = None)
df_mitbih_test = pd.read_csv("../ecg_dataset/mitbih_test.csv", header = None)

# print shapes of the dataframes
print("The shape of the mitbih_train is : ", df_mitbih_train.shape)
print("The shape of the mitbih_test is : ", df_mitbih_test.shape)

The shape of the mitbih_train is :  (87554, 188)
The shape of the mitbih_test is :  (21892, 188)


# Build model for classification

Configure training and testing sets. 

In [4]:
X_train, y_train = df_mitbih_train.iloc[: , :-1], df_mitbih_train.iloc[: , -1]

X_test, y_test = df_mitbih_test.iloc[:,:-1], df_mitbih_test.iloc[: , -1]

In [5]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
# Create an LSTM RNN model

model = Sequential()
model.add(LSTM(units=128, input_shape=(187,1)))
model.add(Dense(units=5, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', 
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)


2023-06-08 12:52:20.471297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
2737/2737 [==============================] - 268s 97ms/step - loss: 0.6614 - accuracy: 0.8275 - precision: 0.8278 - recall: 0.8250
Epoch 2/10
2737/2737 [==============================] - 300s 110ms/step - loss: 0.6450 - accuracy: 0.8278 - precision: 0.8302 - recall: 0.8252
Epoch 3/10
2737/2737 [==============================] - 320s 117ms/step - loss: 0.6500 - accuracy: 0.8279 - precision: 0.8292 - recall: 0.8266
Epoch 4/10
2737/2737 [==============================] - 321s 117ms/step - loss: 0.6420 - accuracy: 0.8280 - precision: 0.8307 - recall: 0.8251
Epoch 5/10
2737/2737 [==============================] - 321s 117ms/step - loss: 0.5837 - accuracy: 0.8360 - precision: 0.8456 - recall: 0.8251
Epoch 6/10
2737/2737 [==============================] - 321s 117ms/step - loss: 0.3512 - accuracy: 0.9002 - precision: 0.9090 - recall: 0.8909
Epoch 7/10
2737/2737 [==============================] - 322s 118ms/step - loss: 0.3186 - accuracy: 0.9039 - precision: 0.9089 - recall: 0.8986


In [10]:
# Evaluate the model on the test set
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

685/685 [==============================] - 25s 36ms/step - loss: 0.2250 - accuracy: 0.9355 - precision: 0.9390 - recall: 0.9336
Loss: 0.22498847544193268
Accuracy: 0.9355015754699707
Precision: 0.9390333294868469
Recall: 0.9336287379264832


In [11]:
# Predict on the test set
y_pred = np.argmax(model.predict(X_test), axis=-1)

685/685 [==============================] - 24s 36ms/step


In [18]:
# Print classification report
report = classification_report(np.argmax(y_test,axis=1), y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.99      0.96     18118
           1       0.74      0.24      0.36       556
           2       0.81      0.75      0.78      1448
           3       0.00      0.00      0.00       162
           4       0.98      0.86      0.92      1608

    accuracy                           0.94     21892
   macro avg       0.69      0.57      0.60     21892
weighted avg       0.92      0.94      0.93     21892



/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)